In [34]:
# import packages
!pip install seaborn
!pip install mlxtend


import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from mlxtend.frequent_patterns import apriori, fpgrowth
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

1912.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


1921.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [39]:
# set file location
file_loc = Path(r'C:\Users\hruss\OneDrive\Documents\GMU\Repositories\Data_files')
print("file location exists:", file_loc.is_dir())

file location exists: True


In [41]:
# import dataset 

file_loc = Path(r'C:\Users\hruss\OneDrive\Documents\GMU\Repositories\Data_files')
df = pd.read_csv(file_loc/'titanic.csv')
df

,Unnamed: 0,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [85]:
# find out number of rows and columns

num_rows, num_cols = df.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

Number of rows: 891
Number of columns: 16


In [84]:
# find null values

nulls = df.isnull()
print("These are the features with null values, and the count of null values in each.")
print(nulls[nulls > 0].sum())

These are the features with null values, and the count of null values in each.
Unnamed: 0       0
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: object


In [86]:
# Impute missing 'age' values based on another ('who') column, and check

category_means = df.groupby('who')['age'].transform('mean')
df1 = df.copy()
df1['age'] = df1['age'].fillna(category_means)
df1[df1['who'] == 'child'].describe()
print("Statistics for 'age' column where 'who' is 'child':")
print(child_age_stats)

Statistics for 'age' column where 'who' is 'child':
count    83.000000
mean      6.369518
std       4.729063
min       0.420000
25%       2.000000
50%       5.000000
75%       9.500000
max      15.000000
Name: age, dtype: float64


In [87]:
# try a quick association rule to see if there is some pattern to deck = NaN
# first we create a categories dataframe

df_categories = df1[['deck']].copy()
print(df_categories)

    deck
0    NaN
1      C
2    NaN
3      C
4    NaN
..   ...
886  NaN
887    B
888  NaN
889    C
890  NaN

[891 rows x 1 columns]


In [91]:
# then we encode the category columns

df_categories = df1[['class', 'who', 'deck', 'embark_town']].copy()
codes = cat_encoder.fit_transform(df_categories)
names = cat_encoder.get_feature_names_out(input_features=['class', 'who', 'deck', 'embark_town'])
encoded_df = pd.DataFrame(codes.todense(), columns=names)
encoded_df

,class_First,class_Second,class_Third,who_child,who_man,who_woman,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_nan,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,embark_town_nan
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [100]:
# then we clean things up a bit

df_selected = df1[['survived', 'alone']]
df2 = pd.merge(encoded_df, df_selected, left_index=True, right_index=True, how='inner')
df2['alone'] = df2['alone'].replace([True, False], [1, 0]).infer_objects()

print(df2)

     class_First  class_Second  class_Third  who_child  who_man  who_woman  \
0            0.0           0.0          1.0        0.0      1.0        0.0   
1            1.0           0.0          0.0        0.0      0.0        1.0   
2            0.0           0.0          1.0        0.0      0.0        1.0   
3            1.0           0.0          0.0        0.0      0.0        1.0   
4            0.0           0.0          1.0        0.0      1.0        0.0   
..           ...           ...          ...        ...      ...        ...   
886          0.0           1.0          0.0        0.0      1.0        0.0   
887          1.0           0.0          0.0        0.0      0.0        1.0   
888          0.0           0.0          1.0        0.0      0.0        1.0   
889          1.0           0.0          0.0        0.0      1.0        0.0   
890          0.0           0.0          1.0        0.0      1.0        0.0   

     deck_A  deck_B  deck_C  deck_D  deck_E  deck_F  deck_G  de

/tmp/ipykernel_98/1526525248.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2['alone'] = df2['alone'].replace([True, False], [1, 0]).infer_objects()


In [104]:
# then we do some association rule mining using apriori and print the results

results = apriori(df2, min_support = .2)
results = results[results['itemsets'].apply(lambda x: len(x)) > 2]
results = results[results['itemsets'].apply(lambda x: 'deck_nan' in x)]
results

Empty DataFrame
Columns: [support, itemsets]
Index: []


/tmp/ipykernel_98/2663842311.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2_bool = df2.applymap(lambda x: True if x != 0 else False)


In [107]:
# let's look at a subset of the data we suspect is most closely related to deck = NaN
# we want to filter to: who = man, embark town = southampton, alone = true and class_third = true
# we want the resulting table to just show the counts of each variable after filtering

df2[(df2['who_man'] = True)].sum()

KeyError: 'embark_town_southampton'

In [109]:
# join df1 and df2 for analysis like clustering, correlation, and pca

df3 = pd.concat([df1[['age', 'fare']], df2], axis=0)
df3

,age,fare,class_First,class_Second,class_Third,who_child,who_man,who_woman,deck_A,deck_B,...,deck_E,deck_F,deck_G,deck_nan,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,embark_town_nan,survived,alone
0,22.0,7.2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,71.2833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26.0,7.9250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.0,53.1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35.0,8.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,NaN,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
887,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
888,NaN,NaN,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
889,NaN,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [111]:
# normalize age and fare, so they don't bias our clustering and correlation

cat_columns = ['class', 'who', 'deck', 'embark_town']
df_selected = df_categories[cat_columns]
df_selected = pd.DataFrame(df_selected)
codes = cat_encoder.fit_transform(df_selected)
names = cat_encoder.get_feature_names_out(input_features=cat_columns)
encoded_df = pd.DataFrame(codes.todense(), columns=names)
encoded_df

,class_First,class_Second,class_Third,who_child,who_man,who_woman,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_nan,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,embark_town_nan
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [81]:
# discover correlations 

corr_matrix = df3.corr()
plt.figure(figsize=(15, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

NameError: name 'df3' is not defined